1-make sure every data sample is a json 

2-get rid of the timestamp 

3-convert it to a csv 

In [14]:
import os
import pandas as pd

def convert_csv_to_json(directory):
    # Check if the directory exists
    if not os.path.exists(directory):
        print(f"Directory '{directory}' does not exist.")
        return
    
    # Iterate through files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            # Read CSV file into DataFrame
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            
            # Convert DataFrame to JSON
            json_data = df.to_json(orient='records')
            
            # Write JSON data to file
            json_filename = os.path.splitext(filename)[0] + ".json"
            json_filepath = os.path.join(directory, json_filename)
            with open(json_filepath, 'w') as json_file:
                json_file.write(json_data)
            
            print(f"Converted '{filename}' to '{json_filename}'.")

# Specify the directory containing CSV files
csv_directory = 'data'

# Call the function to convert CSV files to JSON
convert_csv_to_json(csv_directory)


In [ ]:
#timestamp = datetime.datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")

In [1]:
pip install fastapi

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: fastapi in c:\users\1thea\appdata\local\programs\python\python310\lib\site-packages (0.110.0)




[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pytrends

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from pytrends.request import TrendReq
from fastapi import FastAPI
import pandas as pd
import time
import requests
#Interest_rate_Credit&lending : failed retry later 
url = 'https://trends.google.com/trends/'
response = requests.get(url, timeout=10)

app = FastAPI()

country_list = ['AT', 'BE', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'GR', 'ES',
                'FI', 'FR', 'IE', 'IS', 'IT', 'LU', 'ME', 'NL', 'NO', 'PL', 'PT', 'SE', 'SI']

def obtener_datos_google_trends_mensuales(palabra_clave, paises=country_list, fecha_inicio='2014-01-01', fecha_fin='2024-04-05'):
    def retry_obtener_datos():
        try:
            return obtener_datos_google_trends_mensuales(palabra_clave, paises, fecha_inicio, fecha_fin)
        except Exception as e:
            print(f"Error al obtener datos de Google Trends: {e}")
            print("Reintentando en 60 segundos...")
            time.sleep(60)
            return retry_obtener_datos()

    try:
        pytrends = TrendReq(hl='es', tz=360)
        trends_data = {}
        for pais in paises:
            try:
                pytrends.build_payload(kw_list=[palabra_clave], timeframe=f'{fecha_inicio} {fecha_fin}', geo=pais)
                data = pytrends.interest_over_time()
                if data.empty:
                    raise ValueError(f"No hay datos disponibles para la palabra clave y país '{pais}'.")
                data_mensual = data.resample('M').mean()
                trends_data[pais] = data_mensual.to_dict()
            except Exception as e:
                print(f"Error al obtener datos de Google Trends para el país '{pais}': {e}")
                # Retry obtaining data if an error occurs for a country
                trends_data[pais] = retry_obtener_datos()
        return trends_data
    except Exception as e:
        print(f"Error al obtener datos de Google Trends: {e}")
        return retry_obtener_datos()

@app.get("/google-trends/{topic}")
def google_trends_handler(topic: str, fecha_inicio: str = '2014-01-01', fecha_fin: str = '2024-03-31'):
    trends_data = obtener_datos_google_trends_mensuales(topic, country_list, fecha_inicio, fecha_fin)
    return trends_data

palabra_clave = "Affordable housing "
datos12 = obtener_datos_google_trends_mensuales(palabra_clave)
datos12


Error al obtener datos de Google Trends para el país 'IS': No hay datos disponibles para la palabra clave y país 'IS'.
Error al obtener datos de Google Trends para el país 'IS': No hay datos disponibles para la palabra clave y país 'IS'.
Error al obtener datos de Google Trends para el país 'IS': No hay datos disponibles para la palabra clave y país 'IS'.
Error al obtener datos de Google Trends para el país 'IS': No hay datos disponibles para la palabra clave y país 'IS'.
Error al obtener datos de Google Trends para el país 'IS': No hay datos disponibles para la palabra clave y país 'IS'.
Error al obtener datos de Google Trends para el país 'IS': No hay datos disponibles para la palabra clave y país 'IS'.
Error al obtener datos de Google Trends para el país 'IS': No hay datos disponibles para la palabra clave y país 'IS'.
Error al obtener datos de Google Trends para el país 'IS': No hay datos disponibles para la palabra clave y país 'IS'.
Error al obtener datos de Google Trends para el 

KeyboardInterrupt: 